In [40]:
import numpy as np
from matplotlib import pyplot as plt
import time

from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [41]:
my_sizeNtimes=[]
sK_sizeNtimes=[]
d1=np.random.rand(100,2)
d2=np.random.rand(800,2)
d3=np.random.rand(2000,2)
d4=np.random.rand(5000,2)

In [42]:
c1=np.random.choice([-1,1],d1.shape[0]).reshape(-1,1)
c2=np.random.choice([-1,1],d2.shape[0]).reshape(-1,1)
c3=np.random.choice([-1,1],d3.shape[0]).reshape(-1,1)
c4=np.random.choice([-1,1],d4.shape[0]).reshape(-1,1)


In [43]:
# hard-coded SVM
def mySVM(X,y):    
    C = 100
    m,n = X.shape
    Xy = y * X
    H = np.dot(Xy , Xy.T)

    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt_matrix(y.T*1.0)
    b = cvxopt_matrix(np.zeros(1))
    
    start=time.time()
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    end=time.time()

    lambdas = np.array(sol['x'])
    w = np.matmul((y * lambdas).T,X).reshape(-1,1)
    Sv = ((lambdas>0.09999) ).flatten()
    w0 = np.mean(y[Sv] - np.dot(X[Sv], w))
    my_sizeNtimes.append([X.shape[0],(end-start)*10**3])
    return((end-start)*10**3, w, w0 )


In [44]:
#sklearn SVM
def skSvm(X,y):
    starts = time.time()
    model = SVC(C = 100, kernel = 'linear')
    model.fit(X, y.reshape(-1)) 
    ends = time.time()
    sK_sizeNtimes.append([X.shape[0],(ends-starts)*10**3])

    return((ends-starts)*10**3 ,model.coef_, model.intercept_)


In [45]:
mySVM(d1,c1)
skSvm(d1,c1)

mySVM(d2,c2)
skSvm(d2,c2)

mySVM(d3,c3)
skSvm(d3,c3)

# mySVM(d4,c4)
# skSvm(d4,c4)


     pcost       dcost       gap    pres   dres
 0: -4.2667e+03 -1.2150e+05  1e+05  6e-14  8e-14
 1: -5.3203e+03 -1.0965e+04  6e+03  2e-13  9e-14
 2: -8.9069e+03 -1.0122e+04  1e+03  3e-13  1e-13
 3: -9.2004e+03 -9.5008e+03  3e+02  2e-13  1e-13
 4: -9.2874e+03 -9.3436e+03  6e+01  3e-13  2e-13
 5: -9.3087e+03 -9.3121e+03  3e+00  3e-13  2e-13
 6: -9.3101e+03 -9.3101e+03  4e-02  3e-13  1e-13
 7: -9.3101e+03 -9.3101e+03  4e-04  6e-13  1e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.9958e+04 -6.6405e+05  6e+05  5e-13  5e-13
 1: -4.4707e+04 -8.4622e+04  4e+04  1e-12  4e-13
 2: -7.5037e+04 -8.1946e+04  7e+03  9e-13  9e-13
 3: -7.6225e+04 -7.8297e+04  2e+03  4e-13  8e-13
 4: -7.6795e+04 -7.7159e+04  4e+02  2e-13  8e-13
 5: -7.6933e+04 -7.6992e+04  6e+01  1e-13  8e-13
 6: -7.6951e+04 -7.6971e+04  2e+01  7e-13  8e-13
 7: -7.6959e+04 -7.6962e+04  3e+00  6e-13  9e-13
 8: -7.6960e+04 -7.6961e+04  3e-01  2e-12  9e-13
 9: -7.6960e+04 -7.6960e+04  4e-03  1e-12  8e-1

(66.51473045349121, array([[-0.00023851, -0.00102268]]), array([1.00033644]))

In [46]:
sK_sizeNtimes

[[100, 0.0], [800, 16.15738868713379], [2000, 66.51473045349121]]

In [47]:
# what if no optimal solution found? in cvxopt